In [1]:
from splinter import Browser
from bs4 import BeautifulSoup
import pandas as pd
import time

In [12]:
def init_browser():
    # @NOTE: Replace the path with your actual path to the chromedriver
    executable_path = {"executable_path": "chromedriver"}
    return Browser("chrome", **executable_path, headless=False)

In [13]:
#def scrape():
browser = init_browser()

In [14]:
url = "https://mars.nasa.gov/news/"
browser.visit(url)
time.sleep(2)

In [15]:
news_title = []
news_p = []

In [16]:
html = browser.html
soup = BeautifulSoup(html, "html.parser")
results = soup.find_all("li", class_="slide")
for r in results[0]:
    try: 
        news_title = r.find("div", class_="content_title").get_text()
        news_p = r.find("div", class_="article_teaser_body").get_text()
    except:
        pass
    print(news_title)
    print('')
    print(news_p)

NASA's Perseverance Rover Will Peer Beneath Mars' Surface 

The agency's newest rover will use the first ground-penetrating radar instrument on the Martian surface to help search for signs of past microbial life. 


In [18]:
img_site = "https://www.jpl.nasa.gov/spaceimages/?search=&category=Mars"
browser.visit(img_site)
browser.links.find_by_partial_text('FULL').click()
browser.links.find_by_partial_href('/spaceimages/details').click()
browser.links.find_by_partial_href('/spaceimages/images/largesize/').click()

# https://splinter.readthedocs.io/en/latest/browser.html#verifying-page-url-with-browser-url
featured_image_url = browser.url

# for i in img_url:
#     try:
#         featured_image_url = i.find("figure", class_="lede").find("img")["src"].get_text()
#     except:
#         pass
    

# for img in img_url:
# #     print('page:', x, '-------------')
#     print(img.string)

print(featured_image_url)

https://www.jpl.nasa.gov/spaceimages/images/largesize/PIA08003_hires.jpg


In [ ]:
browser.quit()

In [4]:
facts_url ='https://space-facts.com/mars/'

mars_facts = pd.read_html(facts_url)

In [24]:
facts_table = mars_facts[0]
facts_table.head(9)

,0,1
0,Equatorial Diameter:,"6,792 km"
1,Polar Diameter:,"6,752 km"
2,Mass:,6.39 × 10^23 kg (0.11 Earths)
3,Moons:,2 (Phobos & Deimos)
4,Orbit Distance:,"227,943,824 km (1.38 AU)"
5,Orbit Period:,687 days (1.9 years)
6,Surface Temperature:,-87 to -5 °C
7,First Record:,2nd millennium BC
8,Recorded By:,Egyptian astronomers
